In [1]:
import pandas as pd
import numpy as np
from bson import ObjectId

import keyring
import mysql.connector

import arcgis
from arcgis.gis import GIS
import datetime as dt
import getpass
from IPython.display import HTML

from arcgis.geocoding import geocode
from arcgis.features import Feature, FeatureSet, FeatureLayerCollection, use_proximity, FeatureLayer
from arcgis.features import GeoAccessor, GeoSeriesAccessor
from arcgis.network.analysis import generate_origin_destination_cost_matrix

In [2]:
username = "Glodanale"
password = keyring.get_password("arcgis", "Glodanale")
gis = GIS("https://www.arcgis.com", "Glodanale", password)

In [3]:
def pullDataFromMySQL():
    connection = mysql.connector.connect(
            host = "localhost",
            port = 3306,
            user = 'root',
            database = "expandinghorizons",
            password = "dB79@dG2024!"
    )
    
    cursor = connection.cursor()
    
    query = "SELECT * FROM ei"
    cursor.execute(query)
    
    rows = cursor.fetchall()
    
    df = pd.DataFrame(rows, columns=['_id', 'FirstName', 'LastName', 'ServiceHours', 'Address1', 'City', 'State', 'Zipcode', 'County', 'Longitude', 'Latitude'])
    df["Role"] = "ei"
    
    cursor.close()
    connection.close()
    
    return df

In [ ]:
def pullDataFromMySQL():
    connection = mysql.connector.connect(
            host = "localhost",
            port = 3306,
            user = 'root',
            database = "expandinghorizons",
            password = "dB79@dG2024!"
    )
    
    cursor = connection.cursor()
    
    query = "SELECT * FROM child"
    cursor.execute(query)
    
    rows = cursor.fetchall()
    
    df = pd.DataFrame(rows, columns=['_id', 'FirstName', 'LastName', 'ServiceHours', 'Address1', 'City', 'State', 'Zipcode', 'County', 'Longitude', 'Latitude'])
    df["Role"] = "child"
    
    cursor.close()
    connection.close()
    
    return df

In [4]:
data = pullDataFromMySQL()

In [5]:
print(data)

                         _id  FirstName    LastName  ServiceHours  \
0   65d75a2ce09e67e025e8cd30       Mary  Richardson           0.0   
1   65d75bb0e09e67e025e8cd31    Kirstin      Tipton           1.0   
2   65d75c59e09e67e025e8cd32      Kassi      Knight           0.0   
3   65d75c87e09e67e025e8cd33     Rachel      Taylor           0.0   
4   65d75cb9e09e67e025e8cd34    Brannan     Vanover           0.0   
5   65d75ce1e09e67e025e8cd35     Briana     Ailshie           0.0   
6   65d75d08e09e67e025e8cd36       Dawn     Martino           0.0   
7   65d75d4ce09e67e025e8cd37      Casey      Crumly           0.0   
8   65d75d80e09e67e025e8cd38        Kim      Risner           0.0   
9   65d75da9e09e67e025e8cd39      Lydia        Ugwu           0.0   
10  65d75dd5e09e67e025e8cd3a      Emily    Jennings           0.0   
11  65d75dfae09e67e025e8cd3b      Faith       Ihrig           0.0   
12  65d75e28e09e67e025e8cd3c  Stephanie       Cowan           0.0   

                     Address1    

In [6]:
def combine_address(row):
    return f"{row['Address1']}, {row['City']}, {row['State']} {row['Zipcode']}"

data["Full_Address"] = data.apply(combine_address, axis=1)
featureSetDF = data.copy()
#print(featureSetDF)

featureSetDF = featureSetDF.drop(columns=["ServiceHours", "Address1", "City", "State", "Zipcode", "County"])
featureSetDF = featureSetDF[["_id","FirstName", "LastName", "Full_Address", "Longitude", "Latitude", "Role"]]

#Note: delete the next line for actual run through
featureSetDF2 = featureSetDF.copy()

featureSetDF = featureSetDF.iloc[[0]]
featureSetDF2 = featureSetDF2.iloc[3:6]
featureSetDF2.reset_index(drop=True, inplace=True)

print(type(featureSetDF))
#featureSetDF = featureSetDF.to_frame()
print(type(featureSetDF))
print(featureSetDF)
print(len(featureSetDF))
print(featureSetDF.shape)
print("--------------------------------------------------------------------------------------------------------")
print(type(featureSetDF2))
print(featureSetDF2)

<class 'pandas.core.frame.DataFrame'>
<class 'pandas.core.frame.DataFrame'>
                        _id FirstName    LastName  \
0  65d75a2ce09e67e025e8cd30      Mary  Richardson   

                                       Full_Address  Longitude   Latitude  
0  904 West Hillcrest Drive, Johnson City, TN 37604 -82.372108  36.319485  
1
(1, 6)
--------------------------------------------------------------------------------------------------------
<class 'pandas.core.frame.DataFrame'>
                        _id FirstName LastName  \
0  65d75c87e09e67e025e8cd33    Rachel   Taylor   
1  65d75cb9e09e67e025e8cd34   Brannan  Vanover   
2  65d75ce1e09e67e025e8cd35    Briana  Ailshie   

                                       Full_Address  Longitude   Latitude  
0          502 Nunley Drive, Johnson City, TN 37604 -82.415306  36.305367  
1  1573 Old Bristol Highway, Elizabethton, TN 37643 -82.234230  36.391361  
2         1370 Riverbend Drive, Kingsport, TN 37664 -82.522095  36.509079  


In [35]:
def buildFeatureSet(df):
    features = []

    for index, row in df.iterrows():
        attributes = {"Address": row['Full_Address'], "Status": "unconnected", "OBJECTID": row['_id'], "Role": row['Role']}
        geometry = {"x": row["Longitude"], 'y': row["Latitude"]}
        feature = {"attributes": attributes, "geometry": geometry}
        features.append(feature)
        
    location_json = {
        "features": features,
        "spatialReference": {"wkid": 4326, "lastestWkid": 4326},
        "geometryType": "esriGeometryPoint",
        "fields": [
            {"name": "OBJECTID", "type": "esriFieldTypeString", "alias": "OBJECTID", "length": "50"},
            {"name": "Address", "type": "esriFieldTypeString", "alias": "Address", "length": "100"},
            {"name": "Status", "type": "esriFieldTypeString", "alias": "Status", "length": "50"},
            {"name": "Role", "type": "esriFieldTypeString", "alias": "Role", "length": "50"}
        ]
    }
        
    feature_set = FeatureSet.from_dict(location_json)
    return feature_set

In [8]:
def createFeatureLayer(feature_set):
    feature_layer_item = gis.content.import_data(feature_set, title="Test Feature Layer")
    feature_layer_url = feature_layer_item.url
    print("Feature Layer URL:", feature_layer_url)
    
    return feature_layer_item

In [27]:
def analyzeFeatureLayer(feature_layer_item):
    feature_layer = feature_layer_item.layers[0]
    features = feature_layer.query()

    for feature in features:
        print(feature)
        print("\n")

In [28]:
def process(df):
    feature_set = buildFeatureSet(df)
    feature_layer_item = createFeatureLayer(feature_set)
    analyzeFeatureLayer(feature_layer_item)
    
    return feature_layer_item

In [36]:
#start_layer = process(featureSetDF)
stop_layer = process(featureSetDF2)

Feature Layer URL: https://services5.arcgis.com/ZYK688A64hFRZ5yX/arcgis/rest/services/a4eb5b/FeatureServer
{"geometry": {"x": -9174429.90765625, "y": 4342721.095604265, "spatialReference": {"wkid": 102100, "latestWkid": 3857}}, "attributes": {"FID": 1, "Address": "502 Nunley Drive, Johnson City, TN 37604", "Status": "unconnected", "OBJECTID": "65d75c87e09e67e025e8cd33"}}


{"geometry": {"x": -9154272.613997135, "y": 4354606.561637091, "spatialReference": {"wkid": 102100, "latestWkid": 3857}}, "attributes": {"FID": 2, "Address": "1573 Old Bristol Highway, Elizabethton, TN 37643", "Status": "unconnected", "OBJECTID": "65d75cb9e09e67e025e8cd34"}}


{"geometry": {"x": -9186317.564155225, "y": 4370897.85718025, "spatialReference": {"wkid": 102100, "latestWkid": 3857}}, "attributes": {"FID": 3, "Address": "1370 Riverbend Drive, Kingsport, TN 37664", "Status": "unconnected", "OBJECTID": "65d75ce1e09e67e025e8cd35"}}




In [45]:
def updateDestinations(feature_layer_item, id_value):
    feature_layer = feature_layer_item.layers[0]
    feature_layer_query = feature_layer.query(where=f"OBJECTID = '{id_value}'")
    for feature in feature_layer_query.features:
        feature.attributes["Status"] = "connected"
    feature_layer.edit_features(updates=feature_layer_query.features)
    
updateDestinations(stop_layer, "65d75ce1e09e67e025e8cd35")
analyzeFeatureLayer(stop_layer)

{"geometry": {"x": -9174429.90765625, "y": 4342721.095604265, "spatialReference": {"wkid": 102100, "latestWkid": 3857}}, "attributes": {"FID": 1, "Address": "502 Nunley Drive, Johnson City, TN 37604", "Status": "unconnected", "OBJECTID": "65d75c87e09e67e025e8cd33"}}


{"geometry": {"x": -9154272.613997135, "y": 4354606.561637091, "spatialReference": {"wkid": 102100, "latestWkid": 3857}}, "attributes": {"FID": 2, "Address": "1573 Old Bristol Highway, Elizabethton, TN 37643", "Status": "connected", "OBJECTID": "65d75cb9e09e67e025e8cd34"}}


{"geometry": {"x": -9186317.564155225, "y": 4370897.85718025, "spatialReference": {"wkid": 102100, "latestWkid": 3857}}, "attributes": {"FID": 3, "Address": "1370 Riverbend Drive, Kingsport, TN 37664", "Status": "connected", "OBJECTID": "65d75ce1e09e67e025e8cd35"}}




In [26]:
def buildMatrix(origin, destination)

    items = gis.content.search(query="OD Cost Matrix", item_type="Feature Service")
    for item in items:
        item.delete()

    result = use_proximity.connect_origins_to_destinations(origins_layer=origin,
                                                            destinations_layer=destination,
                                                            origins_layer_route_id_field= "Status",
                                                            destinations_layer_route_id_field="Status",
                                                            output_name="OD Cost Matrix",
                                                            gis=gis)

    print("I completed the function")

    od_cost_matrix_sublayer = FeatureLayer.fromitem(result, layer_id=0)

    od_df = od_cost_matrix_sublayer.query(where='1=1', as_df=True)

    # filter only the required columns
    od_df2 = od_df[['From_OBJECTID','To_OBJECTID','Total_Miles','Total_Minutes', 'From_Role', 'To_Role']]
    print(od_df2)

    return od_df2

{"messageCode": "AO_100096", "message": "The OBJECTID in a6d942 does not have unique values.", "params": {"startLayerRouteIDField": "OBJECTID", "startLayer": "a6d942"}}
{"messageCode": "AO_100080", "message": "ConnectOriginsToDestinations failed."}
Failed to execute (ConnectOriginsToDestinations).
Failed.


Exception: Job failed.

In [ ]:
def pushLocationDataToMySQL(locationDF):
    if locationDF.empty:
        print("Error: locationDF is empty")
    else:
        connection = mysql.connector.connect(
            host = "localhost",
            port = 3306,
            user = 'root',
            database = "expandinghorizons",
            password = "dB79@dG2024!"
        )
    
        cursor = connection.cursor()
    
        print(f"\n\nDatabase connected\n\n")
    
        for index, row in locationDF.iterrows():
            if row.isnull().any():
                print(f"{row['From_OBJECTID']} to {row['To_OBJECTID']} contains null values")
            else:
                pairID1 = row["From_OBJECTID"] + row["To_OBJECTID"]
                pairID2 = row["To_OBJECTID"] + row["From_OBJECTID"]
                
                query = f"SELECT * FROM locationdata WHERE pair_id = {pairID1}"
                cursor.execute(query)
                rows1 = cursor.fetchall()
                
                query = f"SELECT * FROM locationdata WHERE pair_id = {pairID2}"
                cursor.execute(query)
                rows2 = cursor.fetchall()
                
                if rows1 or rows2:
                    print("This pair exists in the database")
                    if rows1:
                        pairID = pairID1
                    elif rows2:
                        pairID = pairID2
                    else:
                        print("Error!!!")
                    
                    updateQuery = ("UPDATE locationdata" 
                                   f"SET driving_distance = {row[Total_Miles]}, driving_time = {row[Total_Minutes]}" 
                                   f"WHERE pair_id = {pairID}")
                    cursor.execute(updateQuery)
                    print(f"{pairID} is updated in the MySQL database")
                    
                else:
                    print("This pair does not exist in database")
                    pairID = pairID1
                    insertQuery = ("INSERT INTO locationdata(pair_id, child_id_1, child_id_2, ei_id_1, ei_id_2, assigned, driving_distance, driving_time) "
                                   "VALUES (%s, %s, %s, %s, %s, %s, %s, %s) ")
                
                    if row['From_Role'] == "child":
                        if row["To_Role"] == "child":
                            cursor.execute(insertQuery, (
                                pairID, row['From_OBJECTID'], row["To_OBJECTID"], None, None, 0, row["Total_Miles"], row["Total_Minutes"]))
                            
                        elif row["To_Role"] == "ei":
                            cursor.execute(insertQuery, (
                                pairID, row['From_OBJECTID'], None, row["To_OBJECTID"], None, 0, row["Total_Miles"], row["Total_Minutes"]))
                            
                    elif row["From_Role"] == "ei":
                        if row["To_Role"] == "child":
                            cursor.execute(insertQuery, (
                                pairID, row['To_OBJECTID'], None, row["From_OBJECTID"], None, 0, row["Total_Miles"], row["Total_Minutes"]))
                        elif row["To_Role"] == "ei":
                            cursor.execute(insertQuery, (
                                pairID, None, None, row['From_OBJECTID'], row["To_OBJECTID"], 0, row["Total_Miles"], row["Total_Minutes"]))

                    print(f"{pairID} is inserted in the MySQL database")
            
        connection.commit()
        cursor.close()
        connection.close()

In [46]:
print(od_df)

   OBJECTID RouteName StartTime EndTime StartTimeUTC EndTimeUTC  Total_Miles  \
0         1     1 - 1       NaT     NaT          NaT        NaT     3.290089   
1         2     1 - 2       NaT     NaT          NaT        NaT    13.811811   
2         3     1 - 3       NaT     NaT          NaT        NaT    19.906688   

   Total_Kilometers  Total_Minutes  TotalWait_Minutes  ...  RouteLayerItemURL  \
0          5.294896       7.917875               <NA>  ...               <NA>   
1            22.228      23.989688               <NA>  ...               <NA>   
2         32.036772      25.676583               <NA>  ...               <NA>   

  OriginOID DestinationOID                                      From_Address  \
0         1              1  904 West Hillcrest Drive, Johnson City, TN 37604   
1         1              2  904 West Hillcrest Drive, Johnson City, TN 37604   
2         1              3  904 West Hillcrest Drive, Johnson City, TN 37604   

   From_Type             From_OBJ

In [47]:
od_df.to_csv("test.csv")

In [21]:
od_df2 = od_df[['From_OBJECTID','To_OBJECTID','Total_Miles','Total_Minutes']]
print(od_df2)

              From_OBJECTID               To_OBJECTID  Total_Miles  \
0  65d75a2ce09e67e025e8cd30  65d75c87e09e67e025e8cd33     3.290089   
1  65d75a2ce09e67e025e8cd30  65d75cb9e09e67e025e8cd34    13.811811   
2  65d75a2ce09e67e025e8cd30  65d75ce1e09e67e025e8cd35    19.906688   

   Total_Minutes  
0       7.917875  
1      23.989688  
2      25.676583  
